In [42]:
import math
import pandas as pd

def main():
    filename = 'titanic-homework.csv'
    data = pd.read_csv(filename)
    del data['PassengerId']
    del data['Name']
    
    nodes = {}  # Liście drzewa decyzyjnego
    
    # Entropia całości
    s = 0
    for v in data['Survived']:
        if v == 1:
            s += 1
    f = len(data['Survived']) - s
    set_entropy = -(s/len(data['Survived'])) * math.log2(s/len(data['Survived'])) - (f/len(data['Survived'])) * math.log2(f/len(data['Survived']))
    
    # Tworzenie liści
    for column in data:
        nodes[column] = create_node(column, data[column], set_entropy, data)
        
    for n in nodes:
        print(nodes[n].name, nodes[n].gain_ratio)
        
        
def create_node(name, col, set_entropy, data):
    sums = {}
    trues = {}
    entropies = {}
    
    # Dla wieku etykiety ręcznie stworzone
    if name == 'Age':
            sums['young'] = 0
            sums['middle'] = 0
            sums['old'] = 0
            trues['young'] = 0
            trues['middle'] = 0
            trues['old'] = 0
    
    # Obliczanie ilości pasażerów z daną etykietą i ilości uratowanych pasażerów z daną etykietą
    for i in range(0, len(col)):
        if name == 'Age':
            if col[i] <= 20:
                label = 'young'
            elif col[i] <= 40:
                label = 'middle'
            else:
                label = 'old'
            sums[label] += 1
            if data['Survived'][i] == 1:
                trues[label] += 1
        else:
            if col[i] in sums.keys():
                sums[col[i]] += 1
            else:
                sums[col[i]] = 1
                trues[col[i]] = 0

            if data['Survived'][i] == 1:
                trues[col[i]] += 1
            
    conditional = 0  # Entropia warunkowa
    intrinsic = 0  # Intrinsic information
    for key in sums:
        entropies[key] = 0
        falses = sums[key] - trues[key]
        if trues[key] != 0:
            entropies[key] -= (trues[key] / sums[key]) * math.log2(trues[key] / sums[key])
        if falses != 0:
            entropies[key] -= (falses / sums[key]) * math.log2(falses / sums[key])

        conditional += (sums[key] / len(col)) * entropies[key]
        
        intrinsic -= (sums[key] / len(col)) * math.log2(sums[key] / len(col))
                                                          
    gain = set_entropy - conditional  # Information gain
    gain_ratio = gain / intrinsic  # Gain ratio
    
    return Node(name, gain_ratio)
    

# Liść
class Node:
    name = ''
    gain_ratio = 0
    next_nodes = []
        
    def __init__(self, name, gain_ratio):
        self.name = name
        self.gain_ratio = gain_ratio
    
if __name__ == '__main__':
    main()

{'young': 29, 'middle': 50, 'old': 21} {'young': 14, 'middle': 19, 'old': 7}
Pclass 0.05960489889898
Sex 0.40323636523376316
Age 0.006262882715023246
SibSp 0.025110189595805712
Parch 0.01464001497707953
Survived 1.0
